# Almost Visual Inertial Odometry

# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import MobileNetV2, mobilenet_v3_small, MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm
from  matplotlib import pyplot as plt
import pandas as pd
import numpy as np

# Constants

In [2]:
SEED = 20197
BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
GAMMA = 0.1   # torch default
LR=0.001
EPOCHS = 2000
EARLY_STOPPING = 20


!chmod +x ./data_setup.sh
!. ./data_setup.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
mkdir: cannot create directory ‘data’: File exists


DOWNLOADING THE DATASET
--2025-06-05 13:34:44--  https://zenodo.org/record/1476931/files/advio-01.zip
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1476931/files/advio-01.zip [following]
--2025-06-05 13:34:45--  https://zenodo.org/records/1476931/files/advio-01.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 199009400 (190M) [application/octet-stream]
Saving to: ‘advio-01.zip’

advio-01.zip        100%[===================>] 189.79M  25.0MB/s    in 8.2s    

2025-06-05 13:3

In [3]:
torch.manual_seed(SEED)
np.random.seed(SEED)

# Device

In [4]:
available_cuda = torch.cuda.is_available()
print(f"Available cuda: {available_cuda}")

device = torch.device("cuda" if available_cuda else "cpu")

Available cuda: False


# Dataset

In [5]:
class AdvioDataset(torch.utils.data.Dataset):
      def __init__(self, frames, inertials, labels, inertials_buffer, choose_split=None, split={'train':0.6, 'val':0.2, 'test':0.2}, shuffle=True):
            if shuffle:
                  shuffled_index = torch.randperm(frames.shape[0])
                  frames = frames[shuffled_index]
                  inertials = inertials[shuffled_index]
                  labels = labels[shuffled_index]
                  inertials_buffer = inertials_buffer[shuffled_index]

            if choose_split is None:
                  self.frames = frames
                  self.inertials = inertials
                  self.labels = labels
                  self.inertials_buffer = inertials_buffer
            else:
                  assert split['train'] + split['val'] + split['test'] == 1
                  length = frames.shape[0]
                  if choose_split == "train":
                        self.frames = frames[:round(split['train']*length)]
                        self.inertials = inertials[:round(split['train']*length)]
                        self.labels = labels[:round(split['train']*length)]
                        self.inertials_buffer = inertials_buffer[:round(split['train']*length)]
                  elif choose_split == "val":
                        self.frames = frames[round(split['train']*length):-round(split['test']*length)]
                        self.inertials = inertials[round(split['train']*length):-round(split['test']*length)]
                        self.labels = labels[round(split['train']*length):-round(split['test']*length)]
                        self.inertials_buffer = inertials_buffer[round(split['train']*length):-round(split['test']*length)]
                  elif choose_split == "test":
                        self.frames = frames[-round(split['test']*length):]
                        self.inertials = inertials[-round(split['test']*length):]
                        self.labels = labels[-round(split['test']*length):]
                        self.inertials_buffer = inertials_buffer[-round(split['test']*length):]
                  else:
                        raise Exception(f"The split name '{choose_split}' doesn't exists")

      def __len__(self):
            return len(self.frames) - 1

      def __getitem__(self, index):
            # load sample of frames
            frame_name = self.frames[index]
            scene_name, frame_name = frame_name.split("_")
            frame_name, file_extension = frame_name.split(".")
            sample_frame = torch.Tensor(np.load(f"./data/{scene_name}/iphone/frames/{scene_name}_{frame_name}.npy", allow_pickle=True))

            # build buffer sample of inertials
            sample_inertials_buffer = self.inertials_buffer[index]

            # load labels
            label_odometry = self.labels[index]

            # load inertials
            label_inertial = self.inertials[index]

            return sample_frame, sample_inertials_buffer, label_odometry, label_inertial


frames = np.load("dataset_frames.npy", allow_pickle=True)
inertials = torch.Tensor(np.load("dataset_inertials.npy", allow_pickle=True))
labels = torch.Tensor(np.load("dataset_labels.npy", allow_pickle=True))
buffer_inertials = torch.Tensor(np.load("dataset_buffer.npy", allow_pickle=True))

train_data = AdvioDataset(frames, inertials, labels, buffer_inertials, "train")
val_data = AdvioDataset(frames, inertials, labels, buffer_inertials, "val")
test_data = AdvioDataset(frames, inertials, labels, buffer_inertials, "test")

del frames
del inertials
del labels
del buffer_inertials

params = {'batch_size': BATCH_SIZE,
          'num_workers': 6,
          'drop_last':True}

train_loader = DataLoader(train_data, **params, shuffle=True)
val_loader = DataLoader(val_data, **params, shuffle=True)
test_loader = DataLoader(test_data, **params)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Model

In [6]:
class ConvNet1D(nn.Module):
  def __init__(self):
    super(ConvNet1D,self).__init__()
    self.conv1 = nn.Conv1d(3,8,3,2)
    torch.nn.init.kaiming_uniform_(self.conv1.weight)

    self.conv2 = nn.Conv1d(8,16,3,2)
    torch.nn.init.kaiming_uniform_(self.conv2.weight)

    self.conv3 = nn.Conv1d(16,32,3,2)
    torch.nn.init.kaiming_uniform_(self.conv3.weight)

    self.conv4 = nn.Conv1d(32,64,3,2)
    torch.nn.init.kaiming_uniform_(self.conv4.weight)

    self.conv5 = nn.Conv1d(64,16,3,1)
    torch.nn.init.kaiming_uniform_(self.conv5.weight)

    self.conv6 = nn.Conv1d(16,3,3,1)
    torch.nn.init.kaiming_uniform_(self.conv6.weight)

    self.fc1 = nn.Linear(3,3)

  def forward(self,x):
    # adding uncertanty
    # random_weights = torch.randn(x.shape)*2 + 1
    # random_weights = random_weights.to(device)
    # x = x * random_weights

    x = F.silu(self.conv1(x))
    x = F.silu(self.conv2(x))
    x = F.silu(self.conv3(x))
    x = F.silu(self.conv4(x))
    x = F.silu(self.conv5(x))
    x = F.silu(self.conv6(x))
    x = x.reshape(x.shape[0], 3)
    x = self.fc1(x)

    return x

In [7]:
class InertialNet(nn.Module):
    def __init__(self):
        super(InertialNet, self).__init__()

        self.conv2d_odometry = MobileNetV2(num_classes=3)
        self.conv2d_inertial = MobileNetV2(num_classes=3)
        self.conv1d_inertial = ConvNet1D()

        self.fc1 = nn.Linear(6,6)
        self.fc2 = nn.Linear(6,3)
        self.fc3 = nn.Linear(3,3)

    def forward(self, x, inertial_buffer):
        x_odometry = self.conv2d_odometry(x)
        x_inertial = self.conv2d_inertial(x)

        x_inertial_unsqueezed = torch.unsqueeze(x_inertial, axis=2)
        x_inertial_buffer = torch.concat([inertial_buffer, x_inertial_unsqueezed], axis=2)
        x_inertial_refined = self.conv1d_inertial(x_inertial_buffer)

        x_odometry = x_odometry.reshape((x.shape[0], -1))
        x_inertial_refined = x_inertial_refined.reshape((x.shape[0], -1))
        x = torch.concat([x_odometry, x_inertial_refined], axis=1)

        x = F.silu(self.fc1(x))
        x = F.silu(self.fc2(x))
        x = self.fc3(x)

        return x, x_inertial, x_inertial_refined

In [8]:
def replace_head(model):
    n_inputs = model.classifier[0].in_features
    classifier = nn.Linear(n_inputs, 100)
    model.classifier[0] = classifier

    n_inputs = model.classifier[3].in_features
    classifier = nn.Linear(100, 3)
    model.classifier[3] = classifier
    return model

In [9]:
class InertialNet2(nn.Module):
    def __init__(self):
        super(InertialNet2, self).__init__()

        self.conv2d_odometry = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
        self.conv2d_odometry = replace_head(self.conv2d_odometry)
        self.conv2d_inertial = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
        self.conv2d_odometry = replace_head(self.conv2d_inertial)
        self.conv1d_inertial = ConvNet1D()

        self.fc1 = nn.Linear(6,6)
        self.fc2 = nn.Linear(6,3)
        self.fc3 = nn.Linear(3,3)

    def forward(self, x, inertial_buffer):
        x_odometry = self.conv2d_odometry(x)
        x_inertial = self.conv2d_inertial(x)

        x_inertial_unsqueezed = torch.unsqueeze(x_inertial, axis=2)
        x_inertial_buffer = torch.concat([inertial_buffer, x_inertial_unsqueezed], axis=2)
        x_inertial_refined = self.conv1d_inertial(x_inertial_buffer)

        x_odometry = x_odometry.reshape((x.shape[0], -1))
        x_inertial_refined = x_inertial_refined.reshape((x.shape[0], -1))
        x = torch.concat([x_odometry, x_inertial_refined], axis=1)

        x = F.silu(self.fc1(x))
        x = F.silu(self.fc2(x))
        x = self.fc3(x)

        return x, x_inertial, x_inertial_refined

# Training

In [10]:
def normalize(data):
  data = data - data.min()
  data = data / data.max()
  return data

In [11]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    history_loss = 0
    history_loss=0
    history_odometry_loss=0
    history_inertial_loss=0
    history_inertial_refined_loss=0

    for batch_idx, (data, inertials_buffer, labels_odometry, labels_inertial) in enumerate(train_loader):
        # data preparation
        data = normalize(data)
        labels_odometry = labels_odometry.mean(axis=1)
        inertials_buffer = inertials_buffer.mean(axis=2)
        inertials_buffer = torch.moveaxis(inertials_buffer, 2, 1)
        labels_inertial = labels_inertial.mean(axis=1)

        data = data.to(device)
        inertials_buffer = inertials_buffer.to(device)
        labels_odometry = labels_odometry.to(device)
        labels_inertial = labels_inertial.to(device)

        #forward
        optimizer.zero_grad()
        out, out_inertial, out_inertial_refined = model(data, inertials_buffer)

        # loss
        odometry_loss = torch.sqrt(F.mse_loss(out, labels_odometry))
        inertial_loss = torch.sqrt(F.mse_loss(out_inertial, labels_inertial))
        inertial_refined_loss = torch.sqrt(F.mse_loss(out_inertial_refined, labels_inertial))
        loss = odometry_loss + inertial_loss + inertial_refined_loss

        # backward
        loss.backward()
        optimizer.step()

        #stats
        history_loss += loss.item()
        history_odometry_loss += odometry_loss.item()
        history_inertial_loss += inertial_loss.item()
        history_inertial_refined_loss += inertial_refined_loss.item()

        if batch_idx%(len(train_loader)//5)==0:
            print(f"\t[# {batch_idx: 4}] train_loss: {loss.item():.6f}, odo_loss: {odometry_loss.item():.6f}, ine_loss: {inertial_loss.item():.6f}, ref_loss: {inertial_refined_loss.item():.6f}")

    history_loss /= len(train_loader) # average epoch loss
    history_odometry_loss /= len(train_loader)
    history_inertial_loss /= len(train_loader)
    history_inertial_refined_loss /= len(train_loader)
    print(f"\ttrain_loss: {history_loss:.6f}, odo_loss: {history_odometry_loss:.6f}, ine_loss: {history_inertial_loss:.6f}, ref_loss: {history_inertial_refined_loss:.6f}")

    return history_loss, history_odometry_loss, history_inertial_loss, history_inertial_refined_loss


In [12]:
def validation(model, device, val_loader, epoch):
    model.train()
    history_loss=0
    history_odometry_loss=0
    history_inertial_loss=0
    history_inertial_refined_loss=0

    for batch_idx, (data, inertials_buffer, labels_odometry, labels_inertial) in enumerate(val_loader):
        # data preparation
        data = normalize(data)
        labels_odometry = labels_odometry.mean(axis=1)
        inertials_buffer = inertials_buffer.mean(axis=2)
        inertials_buffer = torch.moveaxis(inertials_buffer, 2, 1)
        labels_inertial = labels_inertial.mean(axis=1)

        data = data.to(device)
        inertials_buffer = inertials_buffer.to(device)
        labels_odometry = labels_odometry.to(device)
        labels_inertial = labels_inertial.to(device)

        # forward
        out, out_inertial, out_inertial_refined = model(data, inertials_buffer)

        # loss
        odometry_loss = torch.sqrt(F.mse_loss(out, labels_odometry))
        inertial_loss = torch.sqrt(F.mse_loss(out_inertial, labels_inertial))
        inertial_refined_loss = torch.sqrt(F.mse_loss(out_inertial_refined, labels_inertial))
        loss = odometry_loss + inertial_loss + inertial_refined_loss

        #stats
        history_loss += loss.item()
        history_odometry_loss += odometry_loss.item()
        history_inertial_loss += inertial_loss.item()
        history_inertial_refined_loss += inertial_refined_loss.item()

    history_loss /= len(val_loader) # average epoch loss
    history_odometry_loss /= len(val_loader)
    history_inertial_loss /= len(val_loader)
    history_inertial_refined_loss /= len(val_loader)
    print(f"\tval_loss: {history_loss:.6f}, odo_loss: {history_odometry_loss:.6f}, ine_loss: {history_inertial_loss:.6f}, ref_loss: {history_inertial_refined_loss:.6f}")

    return history_loss, history_odometry_loss, history_inertial_loss, history_inertial_refined_loss


In [ ]:
model = InertialNet2().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

h_loss_train = []
h_loss_val = []

early_stopping = {"epoch":0, "best":10**3, "epoch_threshold":EARLY_STOPPING}

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")

    _, loss_train, _, _ = train(model, device, train_loader, optimizer, epoch)
    _, loss_val, _, _ = validation(model, device, val_loader, epoch)

    #populating the history of the loss
    h_loss_train.append(loss_train)
    h_loss_val.append(loss_val)

    # early stopping
    if loss_val < early_stopping["best"]:
        early_stopping["best"] = loss_val
        early_stopping["epoch"] = epoch
        print("\t*** Personal Best ***")

    if epoch - early_stopping["epoch"] > early_stopping["epoch_threshold"]:
        print("\n\tStopped by  early stopping!")
        break

np.savez(f"train_loss_{EPOCHS}", h_loss_train)
np.savez(f"val_loss_{EPOCHS}", h_loss_val)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 32.5MB/s]


Epoch 1/2000
	[#    0] train_loss: 18.533836, odo_loss: 17.135317, ine_loss: 0.672632, ref_loss: 0.725888
	[#  180] train_loss: 22.889975, odo_loss: 22.200052, ine_loss: 0.397464, ref_loss: 0.292457
	[#  360] train_loss: 19.796490, odo_loss: 19.195595, ine_loss: 0.348106, ref_loss: 0.252789
	[#  540] train_loss: 18.910254, odo_loss: 17.973690, ine_loss: 0.703203, ref_loss: 0.233361
	[#  720] train_loss: 19.887917, odo_loss: 18.540279, ine_loss: 1.036166, ref_loss: 0.311471
	[#  900] train_loss: 14.018947, odo_loss: 12.324793, ine_loss: 1.419486, ref_loss: 0.274668
	train_loss: 19.011483, odo_loss: 17.981539, ine_loss: 0.739148, ref_loss: 0.290796
	val_loss: 15.144738, odo_loss: 13.518596, ine_loss: 1.367799, ref_loss: 0.258342
	*** Personal Best ***
Epoch 2/2000
	[#    0] train_loss: 15.028761, odo_loss: 13.851675, ine_loss: 0.946963, ref_loss: 0.230122


# Plotting

In [ ]:
plt.plot(h_loss_train, label="train")
plt.plot(h_loss_val, label="validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()

### Zoomed plot (last 30 epochs)

In [ ]:
plt.plot(np.array(h_loss_train)[-50:], label="train")
plt.plot(np.array(h_loss_val)[-50:], label="validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()